In [2]:
import pandas as pd
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity

MODEL_PATH = "./bert_final"  # Pasta onde você reconstruiu o modelo
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# ==========================================
# 1. SIMULAÇÃO DOS DADOS (O que você tem)
# ==========================================

# Simula a árvore de categorias (df_categ)
data_categ = {
    'FAMILIA': ['Cartões', 'Cartões', 'Cartões', 'Conta Corrente', 'Conta Corrente', 'Crédito Imobiliário'],
    'PRODUTO': ['Crédito', 'Crédito', 'Débito', 'Pacote de Serviços', 'Pix', 'Financiamento'],
    'ASSUNTO': ['Anuidade', 'Compra não reconhecida', 'Falha no Chip', 'Cobrança de Tarifa', 'Pix não enviado', 'Juros Abusivos']
}
df_categ = pd.DataFrame(data_categ)

# Simula as reclamações dos clientes (df_manif)
data_manif = {
    'ID_CLIENTE': [1, 2, 3, 4],
    'TEXTO_CLIENTE': [
        "Estão me cobrando uma anuidade que o gerente disse que seria gratis.",
        "Tentei fazer uma transferência instantanea e o dinheiro saiu da conta mas não chegou.",
        "Tem uma compra na padaria de 50 reais que eu não fiz, quero cancelar meu cartão.",
        "A taxa do meu financiamento da casa está muito alta, quero rever."
    ],
    # Colunas vazias que queremos preencher
    'PRED_FAMILIA': None,
    'PRED_PRODUTO': None,
    'PRED_ASSUNTO': None
}
df_manif = pd.DataFrame(data_manif)

print("--- Dataframe de Categorias (Alvo) ---")
df_categ.head()
print("\n--- Dataframe de Manifestações (Para Classificar) ---")
df_manif[['ID_CLIENTE', 'TEXTO_CLIENTE']]

# ==========================================
# 2. CARREGANDO O BERT (Hugging Face)
# ==========================================
print("\n[1/4] Carregando modelo BERT...")
# Nota: Aqui usamos o caminho da pasta que você reconstruiu ou o nome do modelo
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModel.from_pretrained(MODEL_PATH).to(DEVICE)

# Função para gerar o Embedding (vetor numérico) de um texto
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    # Pega a média da última camada oculta (Mean Pooling) para representar a frase
    return outputs.last_hidden_state.mean(dim=1).numpy()

# ==========================================
# 3. CRIAÇÃO DOS EMBEDDINGS DE REFERÊNCIA
# ==========================================
print("[2/4] Criando 'assinaturas' para a árvore de categorias...")

# TRUQUE: Concatenamos a hierarquia para dar contexto completo ao BERT
# Ex: "Cartões Crédito Anuidade" é mais forte que só "Anuidade"
df_categ['texto_full_hierarquia'] = df_categ.apply(
    lambda x: f"{x['FAMILIA']} {x['PRODUTO']} {x['ASSUNTO']}", axis=1
)

# Geramos uma matriz de embeddings para todas as categorias possiveis
# (Em produção, faça isso em batch/lote para ser rápido)
categ_embeddings = np.vstack([get_embedding(text) for text in df_categ['texto_full_hierarquia']])

# ==========================================
# 4. CLASSIFICAÇÃO (MATCHING)
# ==========================================
print("[3/4] Classificando reclamações dos clientes...")

# Função que encontra a melhor categoria
def classificar_reclamacao(texto_cliente):
    # 1. Gera vetor do cliente
    cliente_emb = get_embedding(texto_cliente)
    
    # 2. Calcula similaridade (Cosseno) contra TODAS as categorias
    similaridades = cosine_similarity(cliente_emb, categ_embeddings)
    
    # 3. Pega o índice da maior similaridade
    melhor_indice = np.argmax(similaridades)
    score_confianca = np.max(similaridades)
    
    # Retorna a linha correspondente do df_categ
    return df_categ.iloc[melhor_indice], score_confianca

# Loop para aplicar em cada linha do df_manif
results = []
for index, row in df_manif.iterrows():
    match, score = classificar_reclamacao(row['TEXTO_CLIENTE'])
    
    # Salvando resultados
    df_manif.at[index, 'PRED_FAMILIA'] = match['FAMILIA']
    df_manif.at[index, 'PRED_PRODUTO'] = match['PRODUTO']
    df_manif.at[index, 'PRED_ASSUNTO'] = match['ASSUNTO']
    df_manif.at[index, 'SCORE_CONFIANCA'] = score

# ==========================================
# 5. RESULTADO FINAL
# ==========================================
print("\n[4/4] Resultado Final:")
pd.set_option('display.max_colwidth', 50) # Para ver o texto melhor
df_manif[['TEXTO_CLIENTE', 'PRED_FAMILIA', 'PRED_PRODUTO', 'PRED_ASSUNTO', 'SCORE_CONFIANCA']]

--- Dataframe de Categorias (Alvo) ---

--- Dataframe de Manifestações (Para Classificar) ---

[1/4] Carregando modelo BERT...
[2/4] Criando 'assinaturas' para a árvore de categorias...
[3/4] Classificando reclamações dos clientes...

[4/4] Resultado Final:


,TEXTO_CLIENTE,PRED_FAMILIA,PRED_PRODUTO,PRED_ASSUNTO,SCORE_CONFIANCA
0,Estão me cobrando uma anuidade que o gerente d...,Conta Corrente,Pacote de Serviços,Cobrança de Tarifa,0.579867
1,Tentei fazer uma transferência instantanea e o...,Cartões,Crédito,Compra não reconhecida,0.565921
2,Tem uma compra na padaria de 50 reais que eu n...,Cartões,Crédito,Compra não reconhecida,0.566331
3,A taxa do meu financiamento da casa está muito...,Crédito Imobiliário,Financiamento,Juros Abusivos,0.532504
